In [10]:
import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 22})

import numpy as np

import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/hitp')

from find_converged import concatenate_files

## Atomic energies with and without MIC for 557 Molecules with 38 valence electrons of QM9

In [11]:
# paths to the compounds
dirs = concatenate_files(['/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/paths_atomic_energies'])

In [12]:
# calculate differences
nuc_charge = []
dist_center = []
diff_alch_pot = []
diff_atomic_en = []
diff_atomic_atomisation_en = []

for compound_path in dirs:
    # compare atomic energies and atomic atomisation energies
    mic_path = compound_path + 'atomic_energies_with_mic.txt'
    no_mic_path = compound_path + 'atomic_energies.txt'
    
    ae_mic = np.loadtxt(mic_path)
    ae_no_mic = np.loadtxt(no_mic_path)
    
    # compare alchemical potential, atomic energy and atomic atomisation energy
    nuc_charge.extend(ae_mic[:, 0])
    dist_center.extend(np.linalg.norm(ae_mic[:, 1:4] - np.array([10,10,10]), axis=1))
    diff_alch_pot.extend(ae_mic[:, 4] - ae_no_mic[:, 4])
    diff_atomic_en.extend(ae_mic[:, 5] - ae_no_mic[:, 5])
    diff_atomic_atomisation_en.extend(ae_mic[:, 6] - ae_no_mic[:, 6])

nuc_charge = np.array(nuc_charge)
dist_center = np.array(dist_center)
diff_alch_pot = np.array(diff_alch_pot)
diff_atomic_en = np.array(diff_atomic_en)
diff_atomic_atomisation_en = np.array(diff_atomic_atomisation_en)

In [7]:
# sort results by element
idx_H = np.where(nuc_charge==1.0)
idx_C = np.where(nuc_charge==6.0)
idx_N = np.where(nuc_charge==7.0)
idx_O = np.where(nuc_charge==8.0)
idc = [idx_H, idx_C, idx_N, idx_O]
labels = ['Z = 1', 'Z = 6', 'Z = 7', 'Z = 8']

In [8]:
# Histograms

fig, ax = plt.subplots(1,3)
for i, idx in enumerate(idc):
    ax[0].hist(diff_alch_pot[idx], bins=200, alpha =0.85, label = labels[i])
    ax[1].hist(diff_atomic_en[idx], bins=200, alpha =0.85, label = labels[i])
    ax[2].hist(diff_atomic_atomisation_en[idx], bins=200, alpha =0.85, label = labels[i])
#     ax[1,1].hist(diff_atomic_atomisation_en[idx], bins=200, alpha =0.5, label = labels[i])
    
ax[0].legend()
ax[1].legend()
ax[2].legend()
ax[0].set_xlabel(r'$\Delta \mu$ (MIC - No MIC) [Ha]')
ax[0].set_ylabel('# Atoms')

ax[1].set_xlabel(r'$\Delta E_{ae}$ (MIC - No MIC) [Ha]')

ax[2].set_xlabel(r'$\Delta E_{aae}$ (MIC - No MIC) [Ha]')

Text(0.5, 0, '$\\Delta E_{aae}$ (MIC - No MIC) [Ha]')

In [144]:
# KDEs

from scipy.stats import gaussian_kde as kde

fig, ax = plt.subplots(1,3)
for i, idx in enumerate(idc):
    kde_alch = kde(diff_alch_pot[idx])
    pos_alch = np.linspace(-0.2, 0.0, 500)
    ax[0].plot(pos_alch, kde_alch.evaluate(pos_alch), label = labels[i])
    
    kde_ae = kde(diff_atomic_en[idx])
    pos_ae = np.linspace(-0.4, 0.0, 500)
    ax[1].plot(pos_ae, kde_ae.evaluate(pos_ae), label = labels[i])
    
    kde_aae = kde(diff_atomic_atomisation_en[idx])
    pos_aae = np.linspace(-0.4, 0.15, 500)
    ax[2].plot(pos_aae, kde_aae.evaluate(pos_aae), label = labels[i])

ax[0].legend()
ax[1].legend()
ax[2].legend()
ax[0].set_xlabel(r'$\Delta \mu$ (MIC - No MIC) [Ha]')
ax[0].set_ylabel('KDE')

ax[1].set_xlabel(r'$\Delta E_{ae}$ (MIC - No MIC) [Ha]')

ax[2].set_xlabel(r'$\Delta E_{aae}$ (MIC - No MIC) [Ha]')

Text(0.5, 0, '$\\Delta E_{aae}$ (MIC - No MIC) [Ha]')

In [18]:
# Distance from Box center histogram

fig, ax = plt.subplots(1,1)
for i, idx in enumerate(idc):
    ax.hist(dist_center[idx], bins=100, label = labels[i])
ax.legend()
ax.set_xlabel('Distance from Box Center')
ax.set_ylabel('# Atoms')

Text(0, 0.5, '# Atoms')

## Comparison of atomic energies with different integration schemes with and without MIC

In [2]:
parent_path = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/integration/'
filenames = ['ae_small_MIC_trapz.txt',
'ae_small_trapz.txt',
'ae_small_cs.txt',
'ae_small_MIC_cs.txt',
'ae_trapz.txt',
'ae_MIC_trapz.txt',
'ae_cs.txt',
'ae_MIC_cs.txt']

integration_results = {}

for i in range(len(filenames)):
    
    integration_results.update({filenames[i][:-4]:np.loadtxt(parent_path + filenames[i], delimiter='\t')})
    
lam_vals_large = np.loadtxt(parent_path+'lam_vals_large.txt', delimiter='\t')
lam_vals_small = np.loadtxt(parent_path+'lam_vals_small.txt', delimiter='\t')

In [3]:
trapz = integration_results['ae_small_trapz']
mic_trapz = integration_results['ae_small_MIC_trapz']

In [4]:
trapz.sum(axis=0)-mic_trapz.sum(axis=0)

array([0.07883857, 0.01724942, 0.07498714, 0.00290845, 0.03503685,
       0.10616818, 0.24685949, 0.02508083, 0.01752502, 0.01803871,
       0.01064804, 0.01118809])

In [5]:
trapz = integration_results['ae_trapz']
mic_trapz = integration_results['ae_MIC_trapz']
trapz.sum(axis=0)-mic_trapz.sum(axis=0)

array([0.07827645, 0.01719286, 0.0748203 , 0.00290157, 0.03498515,
       0.10599883, 0.24639321, 0.02492006, 0.017407  , 0.01792565,
       0.01065053, 0.0111938 ])

In [6]:
nuclei = np.array([[ 6.      , 19.979529, 22.784627, 18.908058],
       [ 6.      , 20.35982 , 19.964565, 18.890009],
       [ 8.      , 22.344159, 18.857926, 18.863323],
       [ 8.      , 18.092893, 18.74945 , 18.905025],
       [ 6.      , 18.285827, 16.053484, 18.887816],
       [ 6.      , 15.726597, 15.015175, 18.906865],
       [ 7.      , 13.735016, 14.125508, 18.920052],
       [ 1.      , 21.813395, 23.718436, 18.8968  ],
       [ 1.      , 18.878631, 23.358359, 17.25609 ],
       [ 1.      , 18.90752 , 23.341078, 20.584704],
       [ 1.      , 19.30598 , 15.408752, 17.207074],
       [ 1.      , 19.337767, 15.389776, 20.541321]])

In [7]:
import matplotlib.cm as cm
from matplotlib.colors import Normalize

# Get a color map
my_cmap = cm.get_cmap('inferno')
 
# Get normalize function (takes data in range [vmin, vmax] -> [0, 1])
my_norm = Normalize(vmin=0, vmax=len(nuclei))
data = [2, 3, 5, 6, 8, 12, 7, 5]


fig, ax = plt.subplots(1,1)
ax.bar(nuclei[:,0], trapz.sum(axis=0)-mic_trapz.sum(axis=0), width = 0.5, color=my_cmap((50,200)), alpha=0.5)

ax.set_xlabel('Nuclear charge')
ax.set_ylabel(r'$\Delta$Atomic energy (no MIC - MIC) [Ha]')

Text(0, 0.5, '$\\Delta$Atomic energy (no MIC - MIC) [Ha]')

In [8]:
trapz.sum(axis=0)-mic_trapz.sum(axis=0)

array([0.07827645, 0.01719286, 0.0748203 , 0.00290157, 0.03498515,
       0.10599883, 0.24639321, 0.02492006, 0.017407  , 0.01792565,
       0.01065053, 0.0111938 ])